In [26]:
import networkx as nx
import json
import math
import heapq # priority queue

In [27]:
class Data:
    def __init__(self):
        self.data = self.import_data()
        self.graph = self.create_graph(self.data)

    @staticmethod
    def import_data():
        with open('../data/stations.json') as file:
            data = json.load(file)
        return data

    @staticmethod
    def create_graph(data):
        g = nx.Graph()
        for s1 in data['stations']:
            g.add_node(s1['name'], coordinates=s1['coordinates'], line=s1['line'])
            for s2, distance in s1['connected_to'].items():
                 g.add_edge(s1['name'], s2, weight=distance)
        return g

    def get_graph(self):
        return self.graph

In [28]:
class Al:
    velocity = 600 # 36 km/h = 600 m/min
    transshipment = 8 # 8 min
    stop_time = 0.5 # 30 s = 0.5 min
    @staticmethod
    def h(graph, node1, node2):
        return math.dist(graph.nodes[node1]['coordinates'],graph.nodes[node2]['coordinates'])

    def get_lines(self, graph, station):
        for s in graph.nodes:
            if station == s:
                lines = graph.nodes[s]['line']
                if isinstance(lines, (list, tuple)):
                    return [str(line) for line in lines]
                else:
                    return [str(lines)]
        return None

    def get_line_between(self, graph, station1, station2):
        lines1 = self.get_lines(graph, station1)
        lines2 = self.get_lines(graph, station2)
        common = set(lines1) & set(lines2)
        return next(iter(common), None)

    def astar_algorithm(self, graph, start_point, end_point):
        open_list = [(0, start_point)] # (f, node)
        visited = {}

        g_acc = {station: float('inf') for station in graph.nodes()}
        g_acc[start_point] = 0

        f_acc = {station: float('inf') for station in graph.nodes()}
        f_acc[start_point] = g_acc[start_point] + self.h(graph, start_point, end_point)

        line_acc = {station: None for station in graph.nodes()}
        line_acc[start_point] = None

        #f_acc = {station: float('inf') for station in graph.nodes()}
        #f_acc[start_point] = g_acc[start_point] + self.h(graph, start_point, end_point)

        #t_acc = {station:float('inf') for station in graph.nodes()}
        #t_acc[start_point] = 0

        while len(open_list) > 0:
            f, current = heapq.heappop(open_list)

            if current == end_point:
                cont = 0
                path = []
                while current in visited:
                    path.append(current)
                    current = visited[current]
                path.append(start_point)
                path.reverse()
                # line = []
                # for s in path:
                #     next_line = graph.nodes[s]['line']
                #     if type(next_line) is not list and (not line or line[-1] != next_line):
                #        line.append(next_line)
                #         cont += 1
                #return path, line, t_acc[end_point] + (cont - 1)        * self.transshipment
                return path, g_acc[end_point]

            for n in graph.neighbors(current):
                # possible_g = g_acc[current] + graph.get_edge_data(current, n)['weight']
                possible_g = g_acc[current] + graph.get_edge_data(current, n)['weight']/self.velocity + self.stop_time
                line_between = self.get_line_between(graph, current, n)
                if line_acc[current] is not None and line_acc[current] != line_between:
                    possible_g += self.transshipment


                if possible_g < g_acc[n]:
                    visited[n] = current
                    g_acc[n] = possible_g
                    f_acc[n] = possible_g + self.h(graph, n, end_point)
                    line_acc[n] = line_between

                    # t_acc[n] = t_acc[current] + (graph.get_edge_data(current, n)['weight'] / self.velocity + 30) / 60 # 30 segundos de parada en cada estación
                    heapq.heappush(open_list, (f_acc[n], n))

        return None, None

In [29]:
g = Data().get_graph()
path, time = Al().astar_algorithm(g, 'Observatorio', 'Universidad')
print('Path: %s\nTime: %.2f min' % (path, time))

Path: ['Observatorio', 'Tacubaya', 'San Pedro de los Pinos', 'San Antonio', 'Mixcoac', 'Insurgentes Sur', 'Hospital 20 de Noviembre', 'Zapata', 'Coyoacán', 'Viveros', 'M.A. De Quevedo', 'Copilco', 'Universidad']
Time: 48.42 min
